# import library

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install wxee rasterio pyshp geemap geopandas shapely

In [ ]:
import xarray as xr
import ee
import itertools
import pickle
import pandas as pd
import wxee
import rasterio
import numpy as np
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping

# import constant

In [ ]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

# import xarray sample

In [ ]:
soil_wc = xr.open_dataset('/content/drive/MyDrive/w/soil_water_content_1979001.nc')
soil_wc

In [ ]:
soil_wc.rio.write_crs("epsg:4326", inplace=True)
soil_wc = soil_wc.sel(time = "1979-01-01")
# soil_wc.rio.to_raster('/content/drive/MyDrive/w/swc_4326.tif')


# GEE

In [ ]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/w/private-key.json"
)

ee.Initialize(credentials)

# get landuse data as xarray

In [ ]:
import geemap

Map = geemap.Map()
Map.add_basemap('HYBRID')

shape = ee.Geometry.Rectangle([LON_MAX, LAT_MIN, LON_MIN, LAT_MAX])

CDL = ee.ImageCollection('USDA/NASS/CDL')

# Crop Mask selection

cropland = CDL.filter(ee.Filter.date('2022-01-01', '2022-12-31')).first().select('cropland').clip(shape)

cultivated = CDL.filter(ee.Filter.date('2022-01-01', '2022-12-31')).first().select('cultivated').clip(shape)

# cropland = CDL.filter(ee.Filter.date('2022-01-01','2022-12-31')).select('cropland').filterBounds()

# type(cropland)
# Map.centerObject(shape, 9)
# Map.addLayer(cropland, {}, 'cropland')
# Map

arr_cropland = cropland.wx.to_xarray(scale=10000)
arr_cultivated = cultivated.wx.to_xarray(scale=10000)

xds_repr_match_cropland = arr_cropland.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.max)
xds_repr_match_cultivated = arr_cultivated.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.max)


In [ ]:
xds_repr_match_cropland.rio.write_crs("epsg:4326", inplace=True)
xds_repr_match_cultivated.rio.write_crs("epsg:4326", inplace=True)

In [ ]:
xds_repr_match_cropland.cropland.data = np.where(xds_repr_match_cropland.cropland.data ==-3.2768e+04 , np.nan, xds_repr_match_cropland.cropland.data)
xds_repr_match_cultivated.cultivated.data = np.where(xds_repr_match_cultivated.cultivated.data ==-3.2768e+04 , np.nan, xds_repr_match_cultivated.cultivated.data)


# merge two dataset

In [ ]:
merged_dataset = xr.merge([xds_repr_match_cropland, xds_repr_match_cultivated])
merged_dataset

In [ ]:
# merged_dataset = merged_dataset.sel(time = "2022-01-01")
# merged_dataset.rio.to_raster('/content/drive/MyDrive/w/crop_4326_1.tif')

# get kc file

In [ ]:

df_1 = pd.read_excel("/content/drive/MyDrive/w/cdl_table.xlsx", sheet_name="Sheet2")
df_1


# map dataframe on landse data

In [ ]:
column_titles = df_1.columns.tolist()
omitted_list = column_titles[1:]

df = merged_dataset.to_dataframe()

for col in omitted_list:

    land_use_dict = dict(zip(df_1['Value'], df_1[col]))

    df[col] = df['cropland'].map(land_use_dict)



df['Plant Date'] = df['Plant Date'].str[:3]


# Convert to numeric month
df['Plant Date'] = df['Plant Date'].apply(lambda x: datetime.strptime(x, "%b").month
                                if isinstance(x, str) else x)


# lll = df['Plant Date'].to_list()

# unique_list = pd.Series(lll).unique()
# type(unique_list[1])

crop_name_da = df.to_xarray()
crop_name_da = crop_name_da.transpose('time', 'y', 'x')
crop_name_da



In [ ]:
crop_name_da.cropland.data = np.where(crop_name_da.cropland.data ==-np.inf, np.nan, crop_name_da.cropland.data)
crop_name_da.cultivated.data = np.where(crop_name_da.cultivated.data ==-np.inf, np.nan, crop_name_da.cultivated.data)

In [ ]:
np.unique(crop_name_da.cultivated)

In [ ]:
np.unique(crop_name_da["cropland"].sel(time = "2022-01-01"))

In [ ]:
shp_dir = r"/content/drive/MyDrive/w/cal/subregions_C2VSim_California.shp"
shapefile_basins = gpd.read_file(shp_dir)

In [ ]:
def mask_xarray(
   array,
   shapefile_basins
):

   shapefile_basins = shapefile_basins.to_crs("epsg:4326")
   shapefile_basins.crs = {'init': 'epsg:4326'}
   array.rio.write_crs("epsg:4326", inplace=True)
   clipped = array.rio.clip(shapefile_basins.geometry.apply(mapping), shapefile_basins.crs)
   # clipped.data = np.where(clipped.data == -9.99900e+03, np.nan, clipped.data)

   return clipped

In [ ]:
land_use = mask_xarray(array = crop_name_da, shapefile_basins = shapefile_basins)
np.unique(land_use.cultivated)

# save xarray as a pikle file

In [ ]:
with open("/content/drive/MyDrive/w/land_use_4326_for_cal.pkl", 'wb') as f:  # open a text file
    pickle.dump(land_use, f) # serialize the list
